In [24]:
import os
import json
import pandas as pd

from datetime import datetime
from utils.config import DRLConfig
from utils.drl_train import training_pipeline, slice_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
df_ret = pd.read_parquet("../data/returns.parquet")
df_prices = pd.read_parquet("../data/prices.parquet")
df_vol = pd.read_parquet("../data/vola.parquet")

To view the logs:
1. Open a terminal or command prompt.
2. Navigate to the directory *containing* the `logs` directory (i.e., the root of this repository).
3. Run the command: `tensorboard --logdir logs/`
4. Open the URL provided by TensorBoard (usually http://localhost:6006/) in your web browser.

You should see experiments named like `PPO_WindowX_AgentY_SeedZ`.

In [26]:
# Create timestamp for this run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create configuration
config = DRLConfig(
    # Window configuration
    n_windows=1,  # 10 in paper
    agents_per_window=5,  # 5 in paper
    base_start_year=2006,
    # Environment parameters
    env_window_size=60,
    transaction_cost=0.0,
    initial_balance=100_000,
    reward_scaling=1.0,
    eta_dsr=1 / 252,
    # Training parameters
    n_envs=10,
    total_timesteps_per_round=7_500_000,  # 7_500_000 in paper
    n_steps_per_env=252 * 3,
    batch_size=1260,
    n_epochs=16,
    gamma=0.9,
    gae_lambda=0.9,
    clip_range=0.25,
    log_std_init=-1.0,
    # Learning rate parameters
    initial_lr=3e-4,
    final_lr=1e-5,
    # Paths
    model_save_dir=f"../models/{timestamp}",
    tensorboard_log_dir=f"../logs/{timestamp}",
)

In [27]:
# Save config as JSON
from dataclasses import asdict
config_dict = asdict(config)
config_json_path = os.path.join(config.model_save_dir, f"config_{timestamp}.json")
os.makedirs(config.model_save_dir, exist_ok=True)
with open(config_json_path, 'w') as f:
    json.dump(config_dict, f, indent=4)
print(f"\nConfiguration saved to: {config_json_path}")


Configuration saved to: ../models/20250611_234904/config_20250611_234904.json


In [28]:
# Run training pipeline
results, backtest_portfolio = training_pipeline(
    drl_config=config, df_prices=df_prices, df_ret=df_ret, df_vol=df_vol
)

--- Starting Window 1/1 (Train Year Start: 2006) ---
  Train Period: 2006-01-01 to 2010-12-31
  Val Period  : 2011-01-01 to 2011-12-31
  Test Period : 2012-01-01 to 2012-12-31
  Training Agent 1/5 with seed 0...


Output()

    Starting training for 7500000 timesteps...



Training complete. Trained for 7500000 timesteps.
TensorBoard logs for experiment 'PPO_Seed0' saved in directory: ../logs/20250611_234904
    Evaluating agent on validation set...
    Validation Mean Reward: -8.6959
    Agent saved to: ../models/20250611_234904/agent_seed0_valrew-8.70.zip
  Training Agent 2/5 with seed 1...


Output()

...

    Starting training for 7500000 timesteps...



Training complete. Trained for 7500000 timesteps.
TensorBoard logs for experiment 'PPO_Seed3' saved in directory: ../logs/20250611_234904
    Evaluating agent on validation set...
    Validation Mean Reward: -9.3217
    Agent saved to: ../models/20250611_234904/agent_seed3_valrew-9.32.zip
  Training Agent 5/5 with seed 4...


Output()

    Starting training for 7500000 timesteps...



Training complete. Trained for 7500000 timesteps.
TensorBoard logs for experiment 'PPO_Seed4' saved in directory: ../logs/20250611_234904
    Evaluating agent on validation set...
    Validation Mean Reward: -13.1212
    Agent saved to: ../models/20250611_234904/agent_seed4_valrew-13.12.zip
    Running backtest evaluation...


In [29]:
for idx, p in backtest_portfolio.items():
    fname = f"{idx}_portfolio.csv"
    p.get_history().to_csv(os.path.join(config.model_save_dir, fname))

In [30]:
results_filename = f"backtest_results_summary_{timestamp}.csv"
results_save_path = os.path.join(config.model_save_dir, results_filename)

results_df = pd.DataFrame(results)
results_df.to_csv(results_save_path, index=False)
print(f"\nBacktest results summary saved to: {results_save_path}")
print("\nFinal Results DataFrame:")
results_df.head()


Backtest results summary saved to: ../models/20250611_234904/backtest_results_summary_20250611_234904.csv

Final Results DataFrame:


,window,best_agent_path,n_eval_episodes,mean_reward,std_reward,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,Kurtosis,Tail ratio,Daily value at risk (95%),Portfolio turnover (in %),final_portfolio_value_first_episode
0,1,agent_seed0_valrew-8,1,-23.997776,0.0,0.076154,0.074901,0.100707,0.779247,1.011211,0.908507,-0.07531,1.139255,1.188347,-0.06417,0.705649,1.118682,-0.010244,42.38304,107490
